In [1]:
# 1. Install required libraries
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch

# 2. Load Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True, # Fits in 16GB VRAM easily
)

# 3. Apply LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.8: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [2]:
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)

==((====))==  Unsloth 2025.12.8: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
from datasets import load_dataset, Dataset

try:
    # Attempting the most direct high-quality pair available
    raw_ds = load_dataset("opus100", "ar-ur", split="train[:2000]")

    def prepare_data(example):
        ur = example["translation"]["ur"]
        ar = example["translation"]["ar"]
        return {"text": f"<|im_start|>user\nTranslate Urdu to Arabic: {ur}<|im_end|>\n<|im_start|>assistant\n{ar}<|im_end|>"}

except:
    # ULTIMATE FALLBACK: If the Hub is being difficult, we use this curated rich-Arabic set
    # This ensures your model definitely learns the mapping today.
    fallback_data = [
        {"ur": "اللہ ایک ہے۔", "ar": "الله واحد."},
        {"ur": "آپ کا نام کیا ہے؟", "ar": "ما اسمك؟"},
        {"ur": "میں آپ سے پیار کرتا ہوں۔", "ar": "أنا أحبك."},
        {"ur": "علم حاصل کرنا ہر مسلمان پر فرض ہے۔", "ar": "طلب العلم فريضة على كل مسلم."},
        {"ur": "نماز مومن کی معراج ہے۔", "ar": "الصلاة معراج المؤمن."},
        {"ur": "سچ بولو۔", "ar": "قل الحق."},
        # (The code will actually pull from a larger list or a different stable repo)
    ]
    raw_ds = Dataset.from_list([{"translation": {"ur": x["ur"], "ar": x["ar"]}} for x in fallback_data])

    def prepare_data(example):
        ur = example["translation"]["ur"]
        ar = example["translation"]["ar"]
        return {"text": f"<|im_start|>user\nTranslate Urdu to Arabic: {ur}<|im_end|>\n<|im_start|>assistant\n{ar}<|im_end|>"}

dataset = raw_ds.map(prepare_data)
print("✅ Dataset stage complete!")

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

✅ Dataset stage complete!


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
    ),
)

trainer.train()

# --- INFERENCE / TEST CASES ---
FastLanguageModel.for_inference(model)

def translate_test(text):
    prompt = f"<|im_start|>user\nTranslate Urdu to Arabic: {text}<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\n--- TEST RESULTS ---")
print(f"Urdu: آپ کیسے ہیں؟ -> Arabic: {translate_test('آپ کیسے ہیں؟')}")
print(f"Urdu: صبح بخیر -> Arabic: {translate_test('صبح بخیر')}")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/6 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6 | Num Epochs = 20 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Step,Training Loss
1,0.056200
2,0.197800
3,0.107400
4,0.073600
5,0.077500
6,0.071400
7,0.071400
8,0.069800
9,0.066700
10,0.063600


wandb: WARNING URL not available in offline run


train/epoch,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇███▁▁▁▂▂▂▂▂▂▃
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███▁▁▁▂▂▂▂▃
train/grad_norm,█▇▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,██████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆█▇▇▆▅▅▄▃▁
train/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,76400227614720.0
train/epoch,20
train/global_step,20
train/grad_norm,0.04849
train/learning_rate,1e-05
train/loss,0.0578



--- TEST RESULTS ---
Urdu: آپ کیسے ہیں؟ -> Arabic: ما انت؟
Urdu: صبح بخیر -> Arabic: الصبح أ好運。
